In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('../data/processed_data/merged_characteristics.csv')

########create mkt share data for logit
logit_keys = ['market_ids', 'product_ids', 'firm_ids', 'shares', 'prices', 'demand_instruments0']
nonlinear_cols = ['EHBPercentTotalPremium']
linear_cols = [ 'act_value','MetalLevel_Platinum', 'MetalLevel_Silver','Plan Counts']
mkt_cols = [ 'DP05_0015PE' ,'DP05_0069PE','S1701_C01_017E','StateCodeFL'] 
firm_cols =  linear_cols+ nonlinear_cols   
logit_x_cols = firm_cols + mkt_cols + ['DP03_0095E']


############keys for boosted trees...
keys = ['HIOS ID', 'IssuerId', 'County', 'State', 'FIPS County Code', 'Policy County FIPS Code','County Name']
missing_cols = ['DP05_0028PE', 'Number of Consumers with CSR AV of 87%', 
                'Average Monthly Advanced CSR Payment for Consumers with 87%', 
                'DP05_0018PE', 'Number of Consumers with CSR (AV of 73%/87%/94%)', 'Total Number of Consumers',
                'Number of Consumers with CSR AV of 73%', 'Number of Consumers with CSR AV of 94%','FIPS County Code',
                'Average Monthly Advanced CSR Payment for Consumers with 73%', 
                'Average Monthly Advanced CSR Payment for Consumers with 94%', 
                'DP05_0032PE', 'DP05_0004PE', 'County Name']

#setup y
y_cols = ['Ever Enrolled Count']
keys = ['HIOS ID', 'IssuerId', 'County', 'State', 'FIPS County Code', 'Policy County FIPS Code','County Name']

In [3]:
def create_logit_data(df):
    df = df.copy()
    
    #create market data...
    df['shares'] = df['Ever Enrolled Count']/df['DP03_0095E']
    #add logit columns
    df['product_ids'] = df['IssuerId'].astype(str) +  df['County'].astype(str) 

    #demand_instrument0
    MktIds = np.array(pd.get_dummies(df['IssuerId']))
    MktIds2 = (MktIds.T).dot(MktIds)
    dummies_proj = MktIds.dot( np.linalg.inv( MktIds2 ) ).dot( MktIds.T )
    df['demand_instruments0'] = dummies_proj.dot(df['PREMI27']) #average price across markets

   
    #fix problematic columns
    df = df.rename(columns={'Average Monthly Advanced CSR Payment for Consumers with 94%':'csr_pay_94',
                      'Average Monthly Advanced CSR Payment for Consumers with 87%':'csr_pay_87',
                           'Total Number of Consumers':'csr_tot',
                            'Number of Consumers with CSR AV of 94%':'csr_tot_94'
                           ,'PREMI27':'prices', 'County':'market_ids', 'IssuerId':'firm_ids'})
    
    #standardize the cols
    for col in firm_cols:
        df[col] = df[col]/df[col].std()
    df = df[df['shares']!=0]
    
    
    ######### preprocess lasso data
    lasso_x_cols = ['prices']

    for col in df.columns:
        if (col not in y_cols and col not in keys and col not in missing_cols 
            and not 'StateCode' in col and not 'IssuerId' in col and 'csr_' not in col
            and col not in logit_keys and col not in logit_x_cols):
                df[col] = df[col]/df[col].std()
                lasso_x_cols.append(col) 
    
    df = df.fillna(0)
    
    all_cols = list(set(logit_keys + logit_x_cols + y_cols+lasso_x_cols+logit_x_cols))
    clean_df = df[all_cols]
    return sm.add_constant(clean_df),lasso_x_cols


clean_df,lasso_x_cols = create_logit_data(df)

print(clean_df['prices'].mean()),#(1-clean_df['shares'].mean()))
print(clean_df['Ever Enrolled Count'].sum())
print( (clean_df['shares']*clean_df['DP03_0095E']).sum() )

142.14997349144596
9870266.0
9870266.0


/home/erichschulman/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [4]:
num_trials = 5
training_test = []

#initilize folds
np.random.seed()
kf = KFold(n_splits=num_trials,shuffle=True)
folds_indexes = kf.split(clean_df)

for fold_index in folds_indexes:
    np.random.seed()
    X_train = clean_df.iloc[fold_index[0]]
    X_test = clean_df.iloc[fold_index[1]]
    training_test.append( (X_train, X_test) )

In [5]:
logit_mses = []
logit_r2s = []
lasso_mses = []
lasso_r2s = []
weighted_mses = []
weighted_r2s = []

def fit_model(X_train,X_test, logit = True):
    x_cols = None
    y_cols = None
    model = None
    
    if logit:
        print('--------- logit fold results ---------')
        x_cols = logit_x_cols+['const','demand_instruments0']
        y_cols = ['shares']
        model_fit = sm.Logit(X_train[y_cols], X_train[x_cols]).fit(disp=0)
    else:
        print('--------- lasso fold results ---------')
        x_cols = lasso_x_cols+logit_x_cols +['const']
        y_cols = ['Ever Enrolled Count']
        model_fit = sm.OLS(X_train[y_cols], X_train[x_cols]).fit_regularized(method='elastic_net',
                                                                         alpha=250, L1_wt=1.0)

    pred_enrollments = model_fit.predict(X_test[x_cols])    
    if logit:
        pred_enrollments = pred_enrollments*X_test['DP03_0095E']
        
    return pred_enrollments

def compute_metrics(X_test, pred_enrollments):
    error = pred_enrollments - X_test['Ever Enrolled Count']
    mse = float( (error**2).mean() )
    r2 = float( 1 - mse/X_test['Ever Enrolled Count'].var() ) 
    print('r2', r2,'predicted', pred_enrollments.mean())
    print('--------------------------------------')
    return mse,r2


def print_results(mses,r2s):
    mses,r2s = np.array(mses),np.array(r2s)
    enrollments = clean_df['Ever Enrolled Count']
    print( 'mse', np.array(mses).mean() ,'r2', 1 - float( mses.mean()/enrollments.var()) )
    print( 'mse med', np.median(mses) ,'r2 med',r2s.reshape(num_trials,1)[mses == np.median(mses)][0,0] )


for i in range(num_trials):
    X_train, X_test = training_test[i]
    
    print('--------- fold properties ---------')
    print('training', (X_train['shares']*X_train['DP03_0095E']).mean(),
          'test', X_test['Ever Enrolled Count'].mean())
    print('--------------------------------------')
    
    ###########logit##################
    logit_pred = fit_model(X_train,X_test, logit = True)
    logit_mse,logit_r2 = compute_metrics(X_test, logit_pred)
    logit_mses.append(logit_mse)
    logit_r2s.append(logit_r2)
    
    ############ lasso tree ##################
    lasso_pred = fit_model(X_train,X_test, logit = False)
    lasso_mse,lasso_r2 = compute_metrics(X_test, lasso_pred)
    lasso_mses.append(lasso_mse)
    lasso_r2s.append(lasso_r2)
    
    ########## weighted average ###############
    print('--------- weighted fold results ---------')
    #lasso_weight = lasso_mse/(lasso_mse + logit_mse)
    weighted_pred = .5*lasso_pred + .5*logit_pred
    cutoff = weighted_pred.mean()
    weighted_pred = lasso_pred*(weighted_pred  >= cutoff) +  logit_pred*(weighted_pred  <= cutoff)
    weighted_mse,weighted_r2 = compute_metrics(X_test, weighted_pred)
    weighted_mses.append(weighted_mse)
    weighted_r2s.append(weighted_r2)

    
print('\n--------- overall logit results ---------')
print_results(logit_mses,logit_r2s)

print('\n--------- overall lasso results ---------')
print_results(lasso_mses,lasso_r2s)

print('\n--------- overall weighted results ---------')
print_results(weighted_mses,weighted_r2s)

--------- fold properties ---------
training 1259.6837673475752 test 1117.1533665835411
--------------------------------------
--------- logit fold results ---------
r2 0.4550673211127916 predicted 1132.9048455570276
--------------------------------------
--------- lasso fold results ---------
r2 0.5455276699602486 predicted 1151.3746836540913
--------------------------------------
--------- weighted fold results ---------
r2 0.5482256890491501 predicted 1092.9767595716937
--------------------------------------
--------- fold properties ---------
training 1258.7001403399345 test 1121.086034912718
--------------------------------------
--------- logit fold results ---------
r2 0.06597447435043458 predicted 1217.9103634610312
--------------------------------------
--------- lasso fold results ---------
r2 0.3766291461285385 predicted 1260.2337501348277
--------------------------------------
--------- weighted fold results ---------
r2 0.3795593538989184 predicted 1199.6411700461576
-----